In [5]:
from ldap3 import Server, Connection, ALL, SUBTREE
from ldap3.core.exceptions import LDAPException, LDAPBindError
from ldap3.abstract.entry import Entry
import ldap3


In [1]:
from pydantic import BaseModel, Field

In [ ]:
ldapServer = 'ldap://localhost:1389'
ldapBaseDN = 'dc=fairscape,dc=net'
username = 'admin'
adminPassword = 'adminpassword'
adminRDN = f"cn=admin,{ldapBaseDN}"

In [9]:
def connectLDAPServer(
    ldapURI: str,
    userRDN,
    userPassword
) -> Connection:

    try:
        server = Server(ldapURI, get_info=ALL)
        
        # username and password can be configured during openldap setup
        connection = Connection(server,          
                                user=userRDN, 
                                password=userPassword)
        
        bind_response = connection.bind() # Returns True or False 
        return connection
    except LDAPBindError as e:
        connection = e

class LDAPConfig(BaseModel):
    hostname: str
    port: str
    baseDN: str
    adminRDN: str
    adminPassword: str

    def server_uri(self):
        return f'ldap://{self.hostname}:{self.port}'
    
    def connectAdmin(self):
        return connectLDAPServer(
            ldapURI=self.server_uri(),
            userRDN=self.adminRDN,
            userPassword=self.adminPassword
        )
        

In [8]:
def ldapAddFairscapeUser(
    ldapConnection, 
    userCN: str, 
    userSN: str, 
    userGN: str, 
    userMail: str,
    userPassword: str
):

    return ldapAdd(
        ldapConnection,
        f"cn={userCN},ou=users,{ldapBaseDN}",
        entryAttributes={
            "objectClass": "inetOrgPerson",
            "objectClass": "Person",
            "cn": userCN,
            "sn": userSN,
            "gn": userGN,
            "mail": userMail,
            "userPassword": userPassword
        }
    )


def loginUser(
    ldapConnection: ldap3.Connection,
    username: str,
    userPassword: str,
    baseDN: str
) -> bool:

    userDN = f"cn={username},{baseDN}"
    
    return ldapConnection.compare(
        dn=userDN,
        attribute=userPassword,
        value=username
    )

In [ ]:
def ldapSearch(
    ldapConnection: Connection,
    searchFilter: str,
    searchBase: str = 'dc=fairscape,dc=net',
    searchAttributes: List[str] = ['cn', 'uid', 'o']
) -> List[Entry] :
    """ Function for searching ldap entries

    Arguments:
    ldapConnection: (ldap3.Connection)
    searchFilter: (str) ldap filter expressed as string
    searchBase: (str) the base of ldap tree to search
    searchAttributes: (List[str]) the attributes to return from the search

    Example:
    ```
    results = searchLDAP(
        ldapConn, 
        searchFilter='(objectClass=*)', 
        searchBase='dc=fairscape,dc=net', 
        searchAttributes=['cn', 'uid', 'o']
        )
    ```
    
    """
    try:
        ldapConnection.search(
            search_base=searchBase,
            search_filter=searchFilter,
            search_scope=SUBTREE,
            attributes=searchAttributes
        )
        return ldapConnection.entries
    except LDAPException as e:
        return e


def ldapAdd(
    ldapConnection: Connection, 
    distinguishedName: str,
    entryAttributes: Dict[str, str],
):
    """ Generic Function for adding entries to LDAP
    
    Arguments:
    ldapConnection: (ldap3.Connection) connection to ldap server
    distinguishedName: (str) formated distinguished name for entry
    entryAttributes: (Dict[str,str]) attributes to add to the entry
    """
    return ldapConnection.add(distinguishedName, attributes=entryAttributes)




# utility functions
def ldapGetUsers(
    ldapConnection: Connection
) -> List[Entry]:
    """ Function for returning all fairscape users from LDAP
    """
    return ldapSearch(
        ldapConnection, 
        searchFilter='(objectClass=inetOrgPerson)', 
        searchBase='ou=users,dc=fairscape,dc=net',
        searchAttributes=['cn', 'uid', 'o', "memberof"]
    )

def ldapGetGroups(
    ldapConnection: Connection
) -> List[Entry]:
    """ Utility function for returning all fairscape groups from LDAP
    """
    return ldapSearch(
        ldapConnection, 
        searchFilter='(objectClass=groupOfNames)', 
        searchBase='ou=groups,dc=fairscape,dc=net',
        searchAttributes=['cn', 'member', 'o', 'owner']
    )


In [ ]:
# setup fairscape ldap
def ldapSetupOU(ldapConnection)-> bool:
    """
    """
    
    # check that default ou's dont exist
    ouResults = ldapSearch(ldapConnection,
           searchFilter='(objectClass=organizationalUnit)',
           searchBase='dc=fairscape,dc=net',
           searchAttributes=['ou']
          )
    
    def addUserOU():
        return ldapAdd(
            ldapConnection, 
            distinguishedName="ou=users,dc=fairscape,dc=net", 
            entryAttributes={"objectClass": "organizationalUnit", "ou": "users", "description": "organizational unit of fairscape users"}
        )

    def addGroupOU():
        return ldapAdd(
            ldapConnection, 
            distinguishedName="ou=groups,dc=fairscape,dc=net", 
            entryAttributes={"objectClass": "organizationalUnit", "ou": "groups", "description": "groups of fairscape users"}
        )

    def addOrgOU():
        '''
        add organizations to directory
        '''
        return ldapAdd(
            ldapConnection,
            distinguishedName="ou=orgs,dc=fairscape,dc=net",
            entryAttributes={"objectClass": "organizationalUnit", "ou": "orgs", "description": "organizations of fairscape users"}
        )
    
    #addUserOUResult = addUserOU()
    addGroupOUResult = addGroupOU()
    addOrgOUResult = addOrgOU()


    # TODO log
    #print(f"addUserOUResult: {addUserOUResult}")
    print(f"addGroupOUResult: {addGroupOUResult}")
    print(f"addOrgOUResult: {addOrgOUResult}")
    return None